Install all required dependencies via the CLI:

```bash
python3 -m venv .venv && source .venv/bin/activate
sudo apt install ffmpeg
pip3 install pywhispercpp faster-whisper jiwer sentence_transformers

# if GPU inferencing with cuda:
pip3 install nvidia-cublas-cu11 nvidia-cudnn-cu11
export LD_LIBRARY_PATH=`python3 -c 'import os; import nvidia.cublas.lib; import nvidia.cudnn.lib; print(os.path.dirname(nvidia.cublas.lib.__file__) + ":" + os.path.dirname(nvidia.cudnn.lib.__file__))'`
```

Choose the Whisper model identifier that you would like to use.

In [ ]:
desired_whisper_model = input("Whisper model version (e.g., base.en, large):")
gpu_on = input("Inference with GPU (yes or no)?").lower()
inference_solution = int(
    input("Choose one using the number: (1) pywhispercpp or (2) faster-whisper")
)
inference_solution = "cpp" if inference_solution == 1 else "faster"

print(f"Desired Whisper Model: {desired_whisper_model}")
print(f"GPU inferencing: {gpu_on}")
print(f"Selected inference solution: {inference_solution}")

Read in all available audio files from `samples/audio`. Assumes that the audio files all have a matching file within `sample/truth`.

In [ ]:
import os

audio_dir = "samples/audio/"
transcription_dir = "samples/transcription/"
truth_dir = "samples/truth/"

files = os.listdir(audio_dir)

filenames = []
for file in files:
    name, ext = os.path.splitext(file)
    if (ext):
        filenames.append(name)

print(f"Files: {filenames}")

Use the python binding for whisper.cpp to inference whisper GGML. Cross-platform and cross-language.

Use faster-whisper for CTransformers2 inferencing acceleration. Python only,

In [ ]:
if inference_solution == "cpp":
    from pywhispercpp.model import Model

    model = Model(desired_whisper_model, models_dir="./models")

    for filename in filenames:
        audio_file = f"{audio_dir}{filename}.wav"
        print(f"Filename: {audio_file}")
        segments = model.transcribe(audio_file, speed_up=True)

        transcript = ""
        for segment in segments:
            transcript = " ".join([seg.text for seg in segments])

        transcript_file = f"{transcription_dir}{filename}-{inference_solution}-{desired_whisper_model}.txt"
        with open(transcript_file, "w") as f:
            f.write(transcript)

else:
    from faster_whisper import WhisperModel
    import time

    model = WhisperModel(
        desired_whisper_model,
        device="cuda" if gpu_on == "yes" else "cpu",
        compute_type="int8",
        download_root="./models",
    )

    for filename in filenames:
        audio_file = f"{audio_dir}{filename}.wav"
        segments, info = model.transcribe(audio_file)

        print(f"Filename: {audio_file}")
        start = time.perf_counter()
        segments = list(segments)
        end = time.perf_counter()
        print(f"Transcribed in {end - start:0.4f} seconds")

        transcript = ""
        for segment in segments:
            transcript = " ".join([seg.text for seg in segments])

        transcript_file = f"{transcription_dir}{filename}-{inference_solution}-{desired_whisper_model}.txt"
        with open(transcript_file, "w") as f:
            f.write(transcript)

Calculate Word Error Rate (WER) and Word Information Loss (WIL). WER measures word-level accuracy. WIL measures semantic fidelity. WER compares words. WIL compares meaning.

Calculate the cosine-similarity of the paragraphs using a sentence embedding model.

In [ ]:
from jiwer import wer, process_words
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


for filename in filenames:
    truth = ""
    transcript = ""

    transcript_file = f"{transcription_dir}{filename}-{inference_solution}-{desired_whisper_model}.txt"
    with open(transcript_file, "r") as f:
        transcript = f.read()

    truth_file = f"{truth_dir}{filename}.txt"
    with open(truth_file, "r") as f:
        truth = f.read()

    output = process_words(truth, transcript)

    wer = output.wer
    wil = output.wil

    truth_embedding = model.encode(truth, convert_to_tensor=True)
    transcript_embedding = model.encode(transcript, convert_to_tensor=True)

    document_similarity = util.pytorch_cos_sim(
        truth_embedding, transcript_embedding
    ).item()

    print(f"[{filename}] Word Error Rate: {wer}")
    print(f"[{filename}] Word Information Loss: {wil}")
    print(f"[{filename}] (EXPERIMENTAL) Document Similarity: {document_similarity}")